In [24]:
import pandas as pd
import numpy as np

In [25]:
import os
with open("example_sentences.txt", encoding='utf-8') as f:
            lines = f.readlines()


In [26]:
EXAMPLE_SENTENCES = lines

In [47]:
import os
with open("furniture.txt", encoding='utf-8') as f:
            lines = f.readlines()

In [48]:
furnitures = []
labels = []
furniture_id = []
id = 0 
for line in lines:
    array_of_line = line.split()
    length = len(array_of_line)
    furnitures.append(array_of_line[0])
    labels.append("B-FUR")
    furniture_id.append(id)
    if length > 1:
                for k in range(1,length):
                                        furnitures.append(array_of_line[k])
                                        labels.append("I-FUR")
                                        furniture_id.append(id)
    id = id+1
		

In [49]:
furniture_data = pd.DataFrame({
    "f_id":furniture_id,
    "word":furnitures,
    "label":labels
})
furniture_data

,f_id,word,label
0,0,Baby,B-FUR
1,0,lotion,I-FUR
2,1,Baby,B-FUR
3,1,bottle,I-FUR
4,2,Bassinet,B-FUR
...,...,...,...
176,110,table,I-FUR
177,111,Bin,B-FUR
178,112,Trash,B-FUR
179,113,Trash,B-FUR


In [119]:
sentence_id = []
labels = []
words = []

In [120]:
liste = [1,4,2,3,7]
liste.index(4)

1

In [121]:
import random
def func_create_data(furniture_id, example_sentence_remain, id):
    if len(example_sentence_remain)<20:
        example_sentence_remain = EXAMPLE_SENTENCES.copy()
    
    lst = list(range(0,len(example_sentence_remain)))
    random_samples = random.sample(lst,20)

    sample_sentence = example_sentence_remain.copy()

    print(random_samples)
    for t in range(20):
       #print(random_samples[t])
       to_be_deleted_sentence = example_sentence_remain[random_samples[t]]
       to_be_deleted_index = sample_sentence.index(to_be_deleted_sentence)
       del sample_sentence[to_be_deleted_index]


    for i in range(20):
        sntnc = example_sentence_remain[random_samples[i]]
        list_of_words = sntnc.split()

        for k in list_of_words:
            if k == "[MASK]":
                f_data = furniture_data[furniture_data["f_id"]==furniture_id]
                for j in range(len(f_data)):
                    labels.append(f_data.label.iloc[j])
                    sentence_id.append(id)
                    words.append(f_data.word.iloc[j])
            else:
                labels.append("O")
                sentence_id.append(id)
                words.append(k)
        id+=1 
    return sample_sentence,id
            
	
    

In [122]:
furniture_id_list = list(range(furniture_data.f_id.max()+1))
example_sentence_remain = EXAMPLE_SENTENCES.copy()
id = 0

In [123]:
for i in range(furniture_data.f_id.max()+1):
    example_sentence_remain,id = func_create_data(i,example_sentence_remain,id)

[97, 58, 90, 9, 51, 69, 11, 93, 91, 55, 71, 81, 73, 46, 87, 40, 38, 50, 74, 41]
[36, 32, 63, 42, 65, 64, 1, 53, 52, 78, 40, 72, 8, 35, 21, 27, 6, 19, 71, 12]
[44, 19, 52, 5, 42, 31, 21, 23, 4, 6, 49, 54, 32, 50, 34, 11, 2, 0, 57, 55]
[19, 20, 33, 31, 9, 23, 12, 11, 4, 22, 18, 5, 26, 17, 13, 15, 1, 21, 32, 6]
[4, 3, 13, 9, 12, 1, 7, 18, 10, 17, 11, 5, 0, 8, 6, 16, 14, 15, 2, 19]
[27, 44, 56, 73, 41, 84, 39, 90, 2, 87, 91, 35, 60, 82, 14, 25, 21, 3, 83, 68]
[36, 35, 66, 14, 60, 27, 61, 17, 22, 0, 21, 52, 44, 73, 18, 29, 67, 54, 49, 37]
[28, 26, 15, 42, 34, 10, 54, 23, 36, 49, 6, 29, 12, 50, 43, 48, 5, 52, 46, 0]
[20, 22, 29, 36, 3, 28, 21, 32, 17, 11, 4, 0, 8, 25, 5, 7, 37, 14, 10, 26]
[3, 5, 4, 9, 15, 8, 7, 2, 17, 1, 18, 16, 11, 19, 6, 0, 13, 14, 10, 12]
[76, 66, 47, 82, 53, 71, 75, 15, 43, 40, 23, 31, 92, 87, 34, 8, 84, 42, 44, 78]
[32, 37, 70, 62, 30, 25, 27, 56, 47, 28, 65, 26, 64, 36, 4, 78, 60, 18, 42, 74]
[6, 46, 5, 33, 41, 29, 30, 11, 22, 45, 51, 54, 56, 21, 32, 28, 0, 58, 3, 10]

In [118]:
id

2280

In [124]:
data = pd.DataFrame({
    "sentence_id":sentence_id,
    "word": words,
    "label":labels
})
data

,sentence_id,word,label
0,0,Give,O
1,0,me,O
2,0,a,O
3,0,Baby,B-FUR
4,0,lotion,I-FUR
...,...,...,...
31983,2279,point,O
31984,2279,of,O
31985,2279,my,O
31986,2279,living,O


In [125]:
data.to_excel("model_data.xlsx", encoding="utf-8", index=False)

In [126]:
data[data["sentence_id"]==0]

,sentence_id,word,label
0,0,Give,O
1,0,me,O
2,0,a,O
3,0,Baby,B-FUR
4,0,lotion,I-FUR
5,0,with,O
6,0,a,O
7,0,fabric,O
8,0,seat,O
9,0,and,O
